In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import pdb

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #withot batch norm parameters = 6,379,786 accuracy 97.92%
        #with batch norm parameters =  6,383,818 accuracy 98.76%
        #with batch norm and dropout parameters =  6,383,818 accuracy 98.61% (dropout after relu with dropout value .2)
        #with batch norm and dropout parameters =  6,383,818 accuracy 98.74% (dropout after relu with dropout value .1)
        #with batch norm and dropout parameters =  6,383,818 accuracy 99.03% (dropout after relu with dropout value .05)
        #with batch norm and dropout parameters =  6,383,818 accuracy 98.92% (dropout before relu with dropout value .05)
        #with batch norm and dropout parameters =  6,383,818 accuracy 98.95% (dropout after conv with dropout value .05)
        # self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        # self.batch1 = nn.BatchNorm2d(32)
        # self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        # self.batch2 = nn.BatchNorm2d(64)
        # self.pool1 = nn.MaxPool2d(2, 2)
        # self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        # self.batch3 = nn.BatchNorm2d(128)
        # self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        # self.batch4 = nn.BatchNorm2d(256)
        # self.pool2 = nn.MaxPool2d(2, 2)
        # self.conv5 = nn.Conv2d(256, 512, 3)
        # self.batch5 = nn.BatchNorm2d(512)
        # self.conv6 = nn.Conv2d(512, 1024, 3)
        # self.batch6 = nn.BatchNorm2d(1024)
        # self.conv7 = nn.Conv2d(1024, 10, 3)
        """
        Summary from above trail for better accuracy:
        Using Dropuout 0.05 gives better result(compaired to .1 and .2)
        Applying BatchNorm after Conv function.
        Dropuout after Relu activation function.
        """
        p = 0.05
        self.dropuout= nn.Dropout2d(p)
        #conv1 parameters = (3 x 3 x 1 + 1(bias)) x 16 = 160
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input -1, Output=16,  RF=3
        self.batch1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1) #input -16, Output=16, RF=5
        self.batch2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1) #input -16, Output=16, RF=7
        self.batch3 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)               #RF =14
        self.conv4 = nn.Conv2d(16, 16, 3, padding=1)    # input- 16, Output = 16, RF=16
        self.batch4 = nn.BatchNorm2d(16)
        self.conv5 = nn.Conv2d(16, 16, 3, padding=1)   # input- 16, Output = 16, RF=18
        self.batch5 = nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 16, 3, padding=1) # input- 16, Output = 16, RF=20
        self.batch6 = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2)             # RF=40
        self.conv7 = nn.Conv2d(16, 16, 3)            # input- 16, Output = 16, RF=42
        self.batch7 = nn.BatchNorm2d(16)
        self.conv8 = nn.Conv2d(16, 16, 3)           # input- 16, Output = 16, RF=44
        self.batch8 = nn.BatchNorm2d(16)
        self.conv9 = nn.Conv2d(16, 10, 3)             # input- 16, Output = 10, RF=46

    def forward(self, x):
        #pdb.set_trace()
        x = self.pool1(self.dropuout(F.relu(self.batch3(self.conv3(
            self.dropuout(F.relu(self.batch2(self.conv2(
                self.dropuout(F.relu(self.batch1(self.conv1(x)))))))))))))
        x = self.pool2(self.dropuout(F.relu(self.batch6(self.conv6(
            self.dropuout(F.relu(self.batch5(
                self.conv5(self.dropuout(F.relu(self.batch4(self.conv4(x)))))))))))))
        x = self.dropuout(F.relu(self.batch8(self.conv8(self.dropuout(F.relu(self.batch7(self.conv7(x))))))))
        x = self.conv9(x)
        x = x.view(-1, 10)
        # x = self.pool1(self.dropuout(F.relu(self.batch2(self.conv2(self.dropuout(F.relu(self.batch1(self.conv1(x)))))))))
        # x = self.pool2(self.dropuout(F.relu(self.batch4(self.conv4(self.dropuout(F.relu(self.batch3(self.conv3(x)))))))))
        # x = self.dropuout(F.relu(self.batch6(self.conv6(self.dropuout(F.relu(self.batch5(self.conv5(x))))))))
        # x = self.conv7(x)
        # x = x.view(-1, 10)
        return F.log_softmax(x)

In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
         Dropout2d-3           [-1, 16, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           2,320
       BatchNorm2d-5           [-1, 16, 28, 28]              32
         Dropout2d-6           [-1, 16, 28, 28]               0
            Conv2d-7           [-1, 16, 28, 28]           2,320
       BatchNorm2d-8           [-1, 16, 28, 28]              32
         Dropout2d-9           [-1, 16, 28, 28]               0
        MaxPool2d-10           [-1, 16, 14, 14]               0
           Conv2d-11           [-1, 16, 14, 14]           2,320
      BatchNorm2d-12           [-1, 16, 14, 14]              32
        Dropout2d-13           [-1, 16, 14, 14]               0
           Conv2d-14           [-1, 16,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

#for epoch in range(1, 2):
for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0428, Accuracy: 9860/10000 (98.60%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0330, Accuracy: 9895/10000 (98.95%)



  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0182, Accuracy: 9941/10000 (99.41%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0198, Accuracy: 9942/10000 (99.42%)



loss=0.0061161816120147705 batch_id=937: 100%|██████████| 938/938 [00:16<00:00, 57.84it/s]



Test set: Average loss: 0.0167, Accuracy: 9952/10000 (99.52%)

